In [1]:
#importing packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [37]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\PRATIK
[nltk_data]     PANIGRAHI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
import pandas as pd
import numpy as np
data=pd.read_csv(r'C:\Users\PRATIK PANIGRAHI\Desktop\NLP CLASSES\assignment 8\NLP Projects to do\Project_3\Corona_NLP_test.csv')

In [27]:
data

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
5,6,44958,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,Neutral
6,7,44959,NaN,03-03-2020,Voting in the age of #coronavirus = hand sanit...,Positive
7,8,44960,"Geneva, Switzerland",03-03-2020,"@DrTedros ""We cant stop #COVID19 without prot...",Neutral
8,9,44961,NaN,04-03-2020,HI TWITTER! I am a pharmacist. I sell hand san...,Extremely Negative
9,10,44962,"Dublin, Ireland",04-03-2020,Anyone been in a supermarket over the last few...,Extremely Positive


In [34]:
text=data[['OriginalTweet']]

In [35]:
text['index'] = text.index
documents = text

C:\Users\PRATIK PANIGRAHI\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
documents.head()

,OriginalTweet,index
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,1
2,Find out how you can protect yourself and love...,2
3,#Panic buying hits #NewYork City as anxious sh...,3
4,#toiletpaper #dunnypaper #coronavirus #coronav...,4


In [39]:
print(WordNetLemmatizer().lemmatize('watering', pos='v'))

water


In [40]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'jwelled', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,jwelled,jwell
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [41]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [42]:
doc_sample = documents[documents['index'] == 1].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['When', 'I', "couldn't", 'find', 'hand', 'sanitizer', 'at', 'Fred', 'Meyer,', 'I', 'turned', 'to', '#Amazon.', 'But', '$114.97', 'for', 'a', '2', 'pack', 'of', 'Purell??!!Check', 'out', 'how', '', '#coronavirus', 'concerns', 'are', 'driving', 'up', 'prices.', 'https://t.co/ygbipBflMY']


 tokenized and lemmatized document: 
['couldn', 'hand', 'sanit', 'fred', 'meyer', 'turn', 'amazon', 'pack', 'purel', 'check', 'coronavirus', 'concern', 'drive', 'price', 'https', 'ygbipbflmi']


In [45]:
processed_text = documents['OriginalTweet'].map(preprocess)

In [46]:
processed_text

0       [trend, yorker, encount, supermarket, shelv, p...
1       [couldn, hand, sanit, fred, meyer, turn, amazo...
2                       [protect, love, one, coronavirus]
3       [panic, buy, hit, newyork, citi, anxious, shop...
4       [toiletpap, dunnypap, coronavirus, covid_, new...
5       [rememb, time, pay, gallon, regular, angel, pr...
6       [vote, coronavirus, hand, sanit, supertuesday,...
7       [drtedro, stop, covid, protect, healthwork, pr...
8       [twitter, pharmacist, sell, hand, sanit, live,...
9       [supermarket, day, go, normal, shop, night, si...
10      [best, qualiti, couch, unbeliev, price, avail,...
11      [bewar, counterfeit, tri, sell, fake, mask, ch...
12      [panic, food, buy, germani, coronavirus, begin...
13      [covid_, go, groceri, store, turn, clean, supp...
14      [busi, watch, elect, return, brace, covid, out...
15      [airsewa, flyspicejet, provid, webchecin, cust...
16      [precautionari, measur, take, respect, restaur...
17      [stock

# Bag of words on the dataset

In [47]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [48]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 brooklyn
1 coronavirus
2 encount
3 fear
4 foodkick
5 grocer
6 https
7 ivmkmsqdt
8 maxdeliveri
9 onlin
10 pcrlwh


In [49]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [51]:
print(dictionary)

Dictionary(636 unique tokens: ['fear', 'pictur', 'sell', 'shelv', 'shopper']...)


In [62]:

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]
bow_corpus[1]

[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]

In [63]:
bow_doc_4310 = bow_corpus[1]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 5 ("amazon") appears 1 time.
Word 6 ("check") appears 1 time.
Word 7 ("concern") appears 1 time.
Word 8 ("couldn") appears 1 time.
Word 9 ("drive") appears 1 time.
Word 10 ("hand") appears 1 time.
Word 11 ("pack") appears 1 time.
Word 12 ("sanit") appears 1 time.
Word 13 ("turn") appears 1 time.


# TF-IDF

In [64]:

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [65]:
corpus_tfidf = tfidf[bow_corpus]

In [66]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4137498891773347),
 (1, 0.5862549915322198),
 (2, 0.3964646133100739),
 (3, 0.3118514270752582),
 (4, 0.48029221514960924)]


# LDA using Bag of Words

In [67]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [68]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"work" + 0.024*"like" + 0.017*"home" + 0.015*"stay" + 0.015*"water" + 0.013*"buy" + 0.010*"crazi" + 0.010*"retail" + 0.010*"thing" + 0.010*"want"
Topic: 1 
Words: 0.025*"retail" + 0.020*"close" + 0.013*"home" + 0.011*"demand" + 0.011*"school" + 0.011*"long" + 0.010*"week" + 0.010*"time" + 0.010*"suppli" + 0.009*"line"
Topic: 2 
Words: 0.030*"local" + 0.027*"paper" + 0.027*"time" + 0.026*"toilet" + 0.022*"like" + 0.017*"buy" + 0.016*"shelv" + 0.013*"bank" + 0.013*"item" + 0.013*"donat"
Topic: 3 
Words: 0.033*"toilet" + 0.032*"paper" + 0.021*"buy" + 0.018*"hand" + 0.016*"pandem" + 0.015*"mask" + 0.013*"today" + 0.013*"suppli" + 0.012*"sanit" + 0.011*"retail"
Topic: 4 
Words: 0.022*"demand" + 0.018*"suppli" + 0.013*"servic" + 0.012*"work" + 0.011*"market" + 0.011*"help" + 0.011*"drop" + 0.011*"increas" + 0.010*"think" + 0.010*"coronaoutbreak"
Topic: 5 
Words: 0.020*"help" + 0.018*"buy" + 0.018*"time" + 0.018*"come" + 0.016*"coronaoutbreak" + 0.016*"coronapocalyps" +

# LDA using TF-IDF

In [69]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [70]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"local" + 0.013*"hand" + 0.012*"like" + 0.012*"today" + 0.010*"sanit" + 0.009*"paper" + 0.008*"busi" + 0.008*"toilet" + 0.008*"worri" + 0.008*"suppli"
Topic: 1 Word: 0.014*"retail" + 0.011*"busi" + 0.011*"right" + 0.010*"shelv" + 0.010*"consum" + 0.008*"year" + 0.008*"staff" + 0.007*"think" + 0.007*"want" + 0.007*"work"
Topic: 2 Word: 0.013*"mask" + 0.012*"like" + 0.011*"help" + 0.009*"look" + 0.008*"virus" + 0.008*"coronaoutbreak" + 0.008*"face" + 0.008*"local" + 0.008*"corona" + 0.007*"elder"
Topic: 3 Word: 0.010*"panicbuy" + 0.009*"coronapocalyps" + 0.009*"know" + 0.009*"care" + 0.008*"like" + 0.008*"work" + 0.008*"coronaoutbreak" + 0.008*"consum" + 0.007*"stay" + 0.007*"time"
Topic: 4 Word: 0.012*"coronaoutbreak" + 0.011*"paper" + 0.011*"toilet" + 0.009*"suppli" + 0.009*"close" + 0.009*"quarantin" + 0.008*"like" + 0.008*"shelv" + 0.007*"consum" + 0.007*"water"
Topic: 5 Word: 0.012*"paper" + 0.012*"toilet" + 0.012*"like" + 0.011*"buy" + 0.011*"come" + 0.011*"wee

# Classification of the topics

# Performance evaluation by classifying sample document using LDA Bag of Words model

In [72]:
processed_text[2]

['protect', 'love', 'one', 'coronavirus']

In [73]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.774975061416626	 
Topic: 0.020*"help" + 0.018*"buy" + 0.018*"time" + 0.018*"come" + 0.016*"coronaoutbreak" + 0.016*"coronapocalyps" + 0.013*"home" + 0.008*"suppli" + 0.008*"work" + 0.008*"lockdown"

Score: 0.025005269795656204	 
Topic: 0.030*"hand" + 0.019*"help" + 0.016*"like" + 0.014*"sanit" + 0.013*"toilet" + 0.013*"local" + 0.013*"paper" + 0.011*"countri" + 0.010*"know" + 0.010*"elder"

Score: 0.0250034611672163	 
Topic: 0.017*"local" + 0.017*"paper" + 0.015*"toilet" + 0.013*"work" + 0.012*"week" + 0.012*"leav" + 0.011*"test" + 0.011*"think" + 0.010*"custom" + 0.010*"market"

Score: 0.025003183633089066	 
Topic: 0.026*"work" + 0.024*"like" + 0.017*"home" + 0.015*"stay" + 0.015*"water" + 0.013*"buy" + 0.010*"crazi" + 0.010*"retail" + 0.010*"thing" + 0.010*"want"

Score: 0.025003105401992798	 
Topic: 0.022*"demand" + 0.018*"suppli" + 0.013*"servic" + 0.012*"work" + 0.011*"market" + 0.011*"help" + 0.011*"drop" + 0.011*"increas" + 0.010*"think" + 0.010*"coronaoutbreak"

Score

# Performance evaluation by classifying sample document using LDA TF-IDF model¶

In [74]:

for index, score in sorted(lda_model_tfidf[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749654650688171	 
Topic: 0.010*"panicbuy" + 0.009*"coronapocalyps" + 0.009*"know" + 0.009*"care" + 0.008*"like" + 0.008*"work" + 0.008*"coronaoutbreak" + 0.008*"consum" + 0.007*"stay" + 0.007*"time"

Score: 0.02500942535698414	 
Topic: 0.010*"work" + 0.009*"worker" + 0.009*"retail" + 0.009*"demand" + 0.008*"pandem" + 0.007*"surviv" + 0.007*"thank" + 0.007*"time" + 0.007*"paper" + 0.007*"toilet"

Score: 0.02500431053340435	 
Topic: 0.013*"mask" + 0.012*"like" + 0.011*"help" + 0.009*"look" + 0.008*"virus" + 0.008*"coronaoutbreak" + 0.008*"face" + 0.008*"local" + 0.008*"corona" + 0.007*"elder"

Score: 0.02500378154218197	 
Topic: 0.012*"paper" + 0.012*"toilet" + 0.012*"like" + 0.011*"buy" + 0.011*"come" + 0.011*"week" + 0.010*"virus" + 0.010*"today" + 0.009*"corona" + 0.009*"home"

Score: 0.025003762915730476	 
Topic: 0.015*"buy" + 0.011*"stop" + 0.008*"time" + 0.008*"fear" + 0.008*"amazon" + 0.008*"start" + 0.007*"actual" + 0.007*"fresh" + 0.007*"say" + 0.007*"work"

Score: 0.

In [81]:
unseen_document='LetÂ’s ALL look after the less capable in our village and ensure they stay healthy. Bringing shopping to their doors, help with online '
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.47215038537979126	 Topic: 0.021*"buy" + 0.018*"paper" + 0.017*"toilet" + 0.016*"suppli" + 0.014*"stop"
Score: 0.4278135299682617	 Topic: 0.026*"work" + 0.024*"like" + 0.017*"home" + 0.015*"stay" + 0.015*"water"
Score: 0.012506399303674698	 Topic: 0.020*"help" + 0.018*"buy" + 0.018*"time" + 0.018*"come" + 0.016*"coronaoutbreak"
Score: 0.012505967170000076	 Topic: 0.030*"hand" + 0.019*"help" + 0.016*"like" + 0.014*"sanit" + 0.013*"toilet"
Score: 0.012505476363003254	 Topic: 0.017*"local" + 0.017*"paper" + 0.015*"toilet" + 0.013*"work" + 0.012*"week"
Score: 0.012504578568041325	 Topic: 0.022*"demand" + 0.018*"suppli" + 0.013*"servic" + 0.012*"work" + 0.011*"market"
Score: 0.012503928504884243	 Topic: 0.030*"local" + 0.027*"paper" + 0.027*"time" + 0.026*"toilet" + 0.022*"like"
Score: 0.012503789737820625	 Topic: 0.025*"retail" + 0.020*"close" + 0.013*"home" + 0.011*"demand" + 0.011*"school"
Score: 0.012502999976277351	 Topic: 0.033*"toilet" + 0.032*"paper" + 0.021*"buy" + 0.018*"h